## 获取远程服务器数据

In [ ]:
import pymysql
import  pandas as pd

In [ ]:
db = pymysql.connect(host='210.45.212.126', user='lemon', password='Bigdata0217', port=3306)

In [ ]:
cursor = db.cursor()

In [ ]:
sql = "SELECT uid, weibo_cont, create_time, repost_num, comment_num, praise_num FROM `weibo`.`weibo_data` WHERE create_time > '2018-08-24 00:00' and create_time < '2018-09-20 00:00'"
cursor.execute(sql)

In [ ]:
display_columes = ['uid', 'weibo_cont', 'create_time', 'repost_num', 'comment_num', 'praise_num']
res = cursor.fetchall()
data_df = pd.DataFrame(list(res), columns=display_columes)

In [ ]:
sql2 = "SELECT uid,`name`, gender,verify_type, verify_info  FROM `weibo`.`wbuser`"
cursor.execute(sql2)

In [ ]:
res2 = cursor.fetchall()
display_columes2 = ['uid','name', 'gender','verify_type', 'verify_info']
user_df = pd.DataFrame(list(res2), columns=display_columes2)

In [ ]:
res_df = pd.merge(data_df, user_df)
res_df.to_csv('result.csv',index=None)


### 获取本地综合数据

In [ ]:
import pymysql
import  pandas as pd

In [ ]:
db = pymysql.connect(host='127.0.0.1', user='root', password='root', port=3306)

In [ ]:
cursor = db.cursor()

In [ ]:
sql = "SELECT uid,weibo_cont, create_time, repost_num, comment_num, praise_num FROM `weibo`.`weibo_data` WHERE create_time > '2018-08-24 00:00' and create_time < '2018-09-20 00:00'"
cursor.execute(sql)

In [ ]:
display_columes = ['uid' ,'weibo_cont', 'create_time', 'repost_num', 'comment_num', 'praise_num']
res = cursor.fetchall()
data_df = pd.DataFrame(list(res), columns=display_columes)

In [ ]:
sql2 = "SELECT uid,`name`, gender,verify_type, verify_info  FROM `weibo`.`wbuser`"
cursor.execute(sql2)

In [ ]:
res2 = cursor.fetchall()
display_columes2 = ['uid','name', 'gender','verify_type', 'verify_info']
user_df = pd.DataFrame(list(res2), columns=display_columes2)

In [ ]:
data_df.to_csv('../data/weibo_10000.csv')
user_df.to_csv('../data/user_10000.csv')

In [ ]:
res_df = pd.merge(data_df, user_df, how='left',on=['uid'])
print(len(res_df), len(data_df), len(user_df))

### 处理数据

In [ ]:
res_df['create_time'] = pd.to_datetime(res_df['create_time'])

In [ ]:
# 引爆点分析
top_100 = data_df.sort_values(by=['repost_num','comment_num', 'praise_num'], ascending=[False, False, False])[:100]
top_100 = pd.merge(top_100, user_df, how='left')
top_100.to_excel('../data/top_100.xls')
top_100

In [ ]:
# 画出时间曲线
time_group = res_df.groupby(lambda x: res_df['create_time'][x].date()).size()
from matplotlib import pyplot as plt
plt.figure(figsize=(12,6))
plt.plot(time_group.index, time_group.values, label='weibo count')
plt.title('analysis')
plt.legend()
plt.show()

### 处理传播图

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/repost.csv')

In [3]:
import numpy as np
import re
res_2d_arr = []
colums = ['Source', 'Target']
for row in range(len(df)):
    res =  [item.split(':',1)[0].replace('@','') for item in df.iloc[row,:].text.split('//') if re.match(r'@.*:.*', item)]
    if res:
        res = [df.iloc[row,:].uname] + res + ['Super_4ong']
    else:
        res = [df.iloc[row,:].uname] + ['Super_4ong']
    res.reverse()
    for index in range(len(res)-1):
        res_2d_arr.append([res[index], res[index+1]])
res_df = pd.DataFrame(np.array(res_2d_arr), columns=colums)
res_df.to_csv('chuanbo.csv')

### MongoDB 导入数据

In [1]:
import pandas as pd

In [2]:
from pymongo import MongoClient

In [3]:
c = MongoClient()

In [4]:
cursor = c.Projects['PHP'].find()

In [5]:
df = pd.DataFrame(list(cursor))

,Dependents,Forks,Installs,Issues,Language,Stars,Suggesters,Type,Watchers,desc,package-header,requires,source
0,37,1,26654145,NaN,NaN,2120,2,NaN,5,Symfony polyfill for ctype functions,symfony/polyfill-ctype,php: >=5.3.3,https://packagist.org/users/fabpot/
1,1,36,93364,0,NaN,0,0,NaN,10,"A PHP5 library that is fast, easy to learn and...",spoon/library,php: ^7.1,https://packagist.org/users/tijsverkoyen/
2,0,57,6058,0,NaN,7,0,NaN,5,Realistic password strength estimation PHP lib...,mkopinsky/zxcvbn-php,php: ^5.6 || ^7.0|symfony/polyfill-mbstring: >...,https://packagist.org/users/mkopinsky/
3,1,1,42066,0,NaN,4,0,NaN,1,A simple PHP package for sending messages to S...,jeremykenedy/slack,php: >=5.5.0|guzzlehttp/guzzle: ~6.0|~5.0|~4.0...,https://packagist.org/users/jeremykenedy/
4,3,2,36099,0,NaN,0,0,NaN,1,Provides a cleaner API for reading Doctrine an...,tebru/doctrine-annotation-reader,php: >= 7.1|doctrine/annotations: ^1.2|symfony...,https://packagist.org/users/natebrunette/


In [6]:
df.to_csv('../data/qbPHP.csv')